In [1]:
import pandas as pd
import numpy as np

In [ ]:
imdb_data=pd.read_csv('/content/IMDB Dataset.csv')
imdb_data=imdb_data[:5000]
imdb_data.head()

In [3]:
d={"sentiment":{"positive":1,"negative":0}}
imdb_data=imdb_data.replace(d)
print(imdb_data["sentiment"])

0       1
1       1
2       1
3       0
4       1
       ..
4995    0
4996    1
4997    1
4998    0
4999    0
Name: sentiment, Length: 5000, dtype: int64


In [4]:
imdb_data['sentiment'].value_counts()

0    2532
1    2468
Name: sentiment, dtype: int64

###Preprocessing of text data

In [5]:
import re
#Removing the html strips
def strip_html(text):
    return re.sub('(<br\s*/><br\s*/>)|(\-)|(\/)', '', text)
#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('[.;:!\'?,\"()\[\]]', '', text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)

In [6]:
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english'))
def remove(text):
  word_tokens = word_tokenize(text) 
  filtered_sentence = [w for w in word_tokens if not w in stop_words]   
  filtered_sentence = []  
  for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 
  filter=' '.join(filtered_sentence)
  return filter
imdb_data['review']=imdb_data['review'].apply(remove)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
def lemma(text):
  word_tokens = word_tokenize(text)
  lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_tokens])
  return lemmatized_output
imdb_data['review']=imdb_data['review'].apply(lemma)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [9]:
from sklearn.model_selection import train_test_split
X=imdb_data['review']
Y=imdb_data['sentiment']
X1=imdb_data['review']
Y1=imdb_data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40, random_state=42)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, Y1, test_size=0.40, random_state=42)

In [11]:
X_train=imdb_data.review[:3000]
X_test=imdb_data.review[3000:]
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
stop_words = set(stopwords.words('english'))
vectorizer1 = CountVectorizer(binary=False, ngram_range=(1, 3), stop_words=stop_words)
cv_train =vectorizer1.fit_transform(X_train)
cv_test=vectorizer1.transform(X_test)
vectorizer2 =TfidfVectorizer(stop_words=stop_words)
tf_train=vectorizer2.fit_transform(X_train)
tf_test=vectorizer2.transform(X_test)


In [12]:
Y_train=imdb_data.sentiment[:3000]
Y_test=imdb_data.sentiment[3000:]

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

In [15]:
log = LogisticRegression(solver="lbfgs", random_state=42)
rnd = RandomForestClassifier(n_estimators=100, random_state=42)
nvb = MultinomialNB()
voting = VotingClassifier(estimators=[('lr', log), ('rf', rnd),('nv',nvb)],voting='hard')

###Logistic regression

In [21]:
from sklearn.metrics import accuracy_score
model1=log.fit(tf_train, Y_train)
y_pred = model1.predict(tf_test)
print("logistic regression for tf-idf", accuracy_score(Y_test, y_pred))
model11=log.fit(cv_train, Y_train)
y_pred = model1.predict(cv_test)
print("logistic regression for count vectorizer", accuracy_score(Y_test, y_pred))

logistic regression for tf-idf 0.856
logistic regression for count vectorizer 0.8495


###Naive Bayes 

In [22]:
model2=nvb.fit(tf_train, Y_train)
y_pred = model3.predict(tf_test)
print("naive bayes for tf-idf", accuracy_score(Y_test, y_pred))
model21=nvb.fit(cv_train, Y_train)
y_pred = model3.predict(cv_test)
print("naive bayes for count vectorizer", accuracy_score(Y_test, y_pred))

naive bayes for tf-idf 0.846
naive bayes for count vectorizer 0.8575


###Random forest

In [31]:
model61=rnd.fit(tf_train, Y_train)
y_pred = model61.predict(tf_test)
print("random forest for tf-idf", accuracy_score(Y_test, y_pred))
model6=rnd.fit(cv_train, Y_train)
y_pred = model6.predict(cv_test)
print("random forest for count vectorizer", accuracy_score(Y_test, y_pred))

random forest for tf-idf 0.826
random forest for count vectorizer 0.821


###LinearSVC for tf-idf

In [28]:
regularization = [0.01, 0.05, 0.25, 0.5, 1]
accuracy=[]
for c in regularization:
    lr = LinearSVC(C=c)
    lr.fit(tf_train, Y_train)
    accuracy.append(accuracy_score(Y_test, lr.predict(tf_test)))
    print("Accuracy for C=%s: %s" % (c, accuracy_score(Y_test, lr.predict(tf_test))))
# train final model with highest c
highestC = [regularization[i] for i in range(len(regularization)) if accuracy[i] == max(accuracy)][0]
final_model1 = LinearSVC(C = highestC)
final_model1.fit(tf_train, Y_train)
y_pred = final_model1.predict(tf_test)
print("Final Accuracy: %s" % accuracy_score(Y_test, y_pred))

Accuracy for C=0.01: 0.807
Accuracy for C=0.05: 0.8455
Accuracy for C=0.25: 0.8615
Accuracy for C=0.5: 0.8635
Accuracy for C=1: 0.8595
Final Accuracy: 0.8635


###LinearSVC for count vectorizer

In [29]:
regularization = [0.01, 0.05, 0.25, 0.5, 1]
accuracy=[]
for c in regularization:
    lr = LinearSVC(C=c)
    lr.fit(cv_train, Y_train)
    accuracy.append(accuracy_score(Y_test, lr.predict(cv_test)))
    print("Accuracy for C=%s: %s" % (c, accuracy_score(Y_test, lr.predict(cv_test))))
# train final model with highest c
highestC = [regularization[i] for i in range(len(regularization)) if accuracy[i] == max(accuracy)][0]
final_model11 = LinearSVC(C = highestC)
final_model11.fit(cv_train, Y_train)
y_pred = final_model11.predict(cv_test)
print("Final Accuracy: %s" % accuracy_score(Y_test, y_pred))

Accuracy for C=0.01: 0.8475
Accuracy for C=0.05: 0.852
Accuracy for C=0.25: 0.852
Accuracy for C=0.5: 0.8525
Accuracy for C=1: 0.8515
Final Accuracy: 0.8525


###Voting Classifier

In [34]:
voting_clf=VotingClassifier(estimators=[('lr',model11),('rf',model6),('svc',final_model11),('nvb',model21)],voting='hard')
vote=voting_clf.fit(cv_train,Y_train)
y_pred=vote.predict(cv_test)
print("voting classifier for count vectorizer", accuracy_score(Y_test, y_pred))
voting_clf=VotingClassifier(estimators=[('lr',model1),('rf',model61),('svc',final_model1),('nvb',model2)],voting='hard')
vote1=voting_clf.fit(tf_train,Y_train)
y_pred1=vote1.predict(tf_test)
print("voting classifier for tf-idf vectorizer", accuracy_score(Y_test, y_pred1))

voting classifier for count vectorizer 0.86
voting classifier for tf-idf vectorizer 0.86


In [ ]:
import pickle
pickle_out = open("vote.pkl","wb")
pickle.dump(vote, pickle_out)
pickle_out.close()